In [19]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [20]:
from fastai import *
from fastai.vision import *

setup

In [21]:
path = Config.data_path()/'planet'
path.mkdir(parents=True, exist_ok=True)
path

PosixPath('/home/jupyter/.fastai/data/planet')

In [22]:
df = pd.read_csv(path/'train_v2.csv')

In [23]:
tfms = get_transforms(flip_vert=True, max_lighting=0.1, max_zoom=1.05, max_warp=0.)

np.random.seed(0)
src = (ImageItemList.from_csv(path, 'train_v2.csv', folder='train-jpg', suffix='.jpg')
       .random_split_by_pct(0.2)
       .label_from_df(sep=' '))

data = (src.transform(tfms, size=128)
        .databunch().normalize(imagenet_stats))

In [24]:
arch = models.resnet34

acc_025 = partial(accuracy_thresh, thresh=0.25)
f_score = partial(fbeta, thresh=0.25)
learn = create_cnn(data, arch, metrics=[acc_025, f_score])

load learner

In [25]:
learn.load('stage-2-256-rn34-v2-refreeze-to-1');

predictions

In [26]:
idx2class = {v:k for k,v in learn.data.train_ds.y.c2i.items()}
subm_df = pd.read_csv(path/'sample_submission_v2.csv')
subdic = {c1:c2 for c1,c2 in zip(subm_df.image_name,subm_df.tags)}

def set_test_datafolder(learner, test_folder):
    learner.data = (src.add_test_folder(test_folder)
                    #.datasets(ImageMultiDataset)
                    # looks like since last time (9 days ago)
                    # ImageMultiDataset is no longer used in fastai
                    .transform(tfms, size=256)
                    .databunch().normalize(imagenet_stats))

In [27]:
set_test_datafolder(learn, 'test-jpg')
fnames = [f.name.split('.')[0] for f in learn.data.test_ds.x.items]

In [28]:
raw_preds = learn.TTA(ds_type=DatasetType.Test)

Total time: 12:08



In [29]:
preds = copy(raw_preds)

In [30]:
set_test_datafolder(learn, 'test-jpg-additional')
fnames.extend([f.name.split('.')[0] for f in learn.data.test_ds.x.items])

In [31]:
raw_preds = learn.TTA(ds_type=DatasetType.Test)

Total time: 06:19



In [34]:
preds[0].shape, raw_preds[0].shape

(torch.Size([40669, 17]), torch.Size([20522, 17]))

In [35]:
preds_1 = copy(preds); preds_2 = copy(raw_preds)

In [37]:
preds = (torch.cat((preds_1[0],preds_2[0])), torch.cat((preds_1[1],preds_2[1])))

In [39]:
preds[0].shape

torch.Size([61191, 17])

In [12]:
# preds = torch.load('preds_rn34_20181120.pkl')

In [14]:
# len(preds[0])

61191

checking validation set predictions

In [43]:
val_preds = learn.TTA()

Total time: 02:32



In [47]:
vpreds = val_preds[0]

In [50]:
vpreds[0]

tensor([5.7110e-05, 1.4218e-04, 1.6230e-06, 9.9689e-01, 2.1899e-02, 1.0755e-06,
        2.8414e-05, 1.1715e-03, 5.5544e-01, 2.1369e-03, 9.9450e-01, 2.4789e-03,
        1.9168e-05, 9.8931e-01, 1.4712e-02, 5.4618e-04, 3.6194e-01])

In [51]:
torch.exp(vpreds[0])

tensor([1.0001, 1.0001, 1.0000, 2.7098, 1.0221, 1.0000, 1.0000, 1.0012, 1.7427,
        1.0021, 2.7034, 1.0025, 1.0000, 2.6894, 1.0148, 1.0005, 1.4361])

In [52]:
np.log(1.0001)

9.999500033329732e-05

submitting predictions

In [40]:
import datetime
date = str(datetime.date.today()).replace('-','')

In [59]:
th = 0.001

for i,fn in enumerate(fnames):
    subdic[fn] = ' '.join(idx2class[cdx] for cdx in np.where(preds[0][i]>=th)[0])    
subm_col = [subdic[fn] for fn in subm_df.image_name]
subm_df.tags = subm_col

In [60]:
sub_name = f'planet_stage-2-256-rn34-v2-refreeze-to-1_{date}_th{str(th)}.csv'
subm_df.to_csv(path/'submissions/'/sub_name, index=False, )
! kaggle competitions submit -c planet-understanding-the-amazon-from-space -f {path}/submissions/{sub_name} -m "fastai 1.0; 2-stg; RN34; v2; threshold="{th}

100%|██████████████████████████████████████| 10.1M/10.1M [00:01<00:00, 8.75MB/s]
Successfully submitted to Planet: Understanding the Amazon from Space